In [ ]:
!pip install face_recognition
!pip install streamlit pytube spleeter -q
!pip uninstall tensorflow
!pip install tensorflow==2.8

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.19.5 which is incompatible.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 6.15.0 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.34.0 which is incompatible.
google-colab 1.0.0 requires tornado~=5.1.0; python_version >= "3.0", but you have tornado 6.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
Found existing installation: tensorflow 2.5.0
Uninstalling tensorflow-2.5.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_co

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following packages will be REMOVED:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be upgraded:
  libcudnn8
1 upgraded, 0 newly installed, 1 to remove and 47 not upgraded.
Need to get 430 MB of archives.
After this operation, 3,139 MB disk space will be freed.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  libcudnn8 8.1.0.77-1+cuda11.2 [430 MB]
54% [1 libcudnn8 289 MB/430 MB 67%]^C


In [ ]:
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libcudnn8 is already the newest version (8.1.0.77-1+cuda11.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 47 not upgraded.


In [ ]:
%%capture
!gdown 1YjpZFgiGBxVVMexN4Sgdt0L0V4W8B9za
!gdown 11d9LOEBvAn1V9yvOWnDa4UKgT-dor0Zr
!unzip Models.zip

Downloading...
From: https://drive.google.com/uc?id=1YjpZFgiGBxVVMexN4Sgdt0L0V4W8B9za
To: /content/Models.zip
100% 84.2M/84.2M [00:00<00:00, 336MB/s]
Downloading...
From: https://drive.google.com/uc?id=11d9LOEBvAn1V9yvOWnDa4UKgT-dor0Zr
To: /content/cmudict-0.7b
100% 3.87M/3.87M [00:00<00:00, 300MB/s]
Archive:  Models.zip
replace Models/model_accuracy_31_loss_2.4/keras_metadata.pb? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Models/model_accuracy_31_loss_2.4/keras_metadata.pb  
replace Models/model_accuracy_31_loss_2.4/saved_model.pb? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Models/model_accuracy_31_loss_2.4/saved_model.pb  
replace Models/model_accuracy_31_loss_2.4/variables/variables.data-00000-of-00001? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Models/model_accuracy_31_loss_2.4/variables/variables.data-00000-of-00001  
replace Models/model_accuracy_31_loss_2.4/variables/variables.index? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Models/model_

Please run all three cells

In [ ]:
#@title app.py
%%writefile app.py
import streamlit as st
import numpy as np
import pandas as pd
import tempfile
import vidtoimage
import classify
import os

def main():
    st.set_page_config(layout="wide")
    st.title("Lip Reader AI 💋")
    st.write("Upload your file\n- Please have your face and mouth clearly visible and try to exaggerate your lip motions.\n- The longer or higher resolution the video is, the longer it will take to process.\n- Subtitles are white on the to left corner of the video.\n- This may take a while so please be patient 😊")
    st.write("Steps to upload:\n1. Take a video of yourself or get a video.\n2. Upload it to google drive.\n3. Share the file's link and make sure it is set to anyone with the link: 'viewer'.\n4. Paste the link here.")
    user_input = st.text_input("Google drive link....")
    if user_input != None:
        if st.button("Process"):
            dirs_before = os.listdir()
            id = user_input.split("/")[5]
            command = f"gdown --id {id}"
            os.system(command)
            for i in os.listdir():
              if i not in dirs_before:
                video_file = i
            column = st.columns((1,1))
            video = vidtoimage.Vid(video_file)
            with column[0]:
                #Video data
                expander = st.expander("Your Video Data", expanded=True)
                with expander:
                    st.markdown(f"fps: {video.fps}")
                    st.markdown(f"frames: {video.frame_count}")
                    st.markdown(f"lenght: {video.lenght} seconds")
                    st.markdown(f"Time per sequence: {video.timepersequence}")
                ###
            with column[1]:
                my_bar = st.progress(0)
            video.crop(my_bar)
            with column[1]:
                st.markdown("Now you know you might need to brush your teeth more often...")
                st.image(video.frames_cropped[0], width=250)
            classify.predict(video)
            video.create_subs()
            with column[0]:
                st.markdown("Preview...")
                st.image(video.subbed_frames[video.frame_count//2], width=500)
            video.create_vid()
            with open('output.mp4', 'rb') as f:
                st.download_button('Download video', f, file_name="output.mp4")
            os.remove("output.mp4")
            os.remove(video_file)

if __name__ =="__main__":
    main()


Overwriting app.py


In [ ]:
#@title vidtoimage.py
%%writefile vidtoimage.py

import face_recognition
import cv2

def crop(image):
    face_landmarks_list = face_recognition.face_landmarks(image)
    if face_landmarks_list != []:
        ws = []
        hs = []
        for (k, v) in face_landmarks_list[0]['top_lip'] + face_landmarks_list[0]['bottom_lip']:
            ws.append(k)
            hs.append(v)
        [x, y, w, h] = [min(ws), min(hs), max(ws), max(hs)]
        image = image[y:h, x:w]
    return image

class Vid():
    def __init__(self, file):
        self.cap = cv2.VideoCapture(file)
        self.cap.open(file)
        (major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')
        if int(major_ver)  < 3 :
            fps = self.cap.get(cv2.cv.CV_CAP_PROP_FPS)
        else :
            fps = self.cap.get(cv2.CAP_PROP_FPS)

        self.frame_count = int(self.cap.get(cv2. CAP_PROP_FRAME_COUNT))
        self.fps = fps
        self.lenght = self.frame_count/fps
        self.SEQUENCE_LENGHT = 4
        self.timepersequence = self.SEQUENCE_LENGHT/self.fps
        self.frames = []
        self.frames_cropped = []
        self.prediction = {}
        self.phone_timestamp = []
        self.possible_words = []
        self.width = 0
        self.height = 0

    def crop(self, progress_bar):
        run = 0
        while True:
            _, frame = self.cap.read()
            if frame is None:
                break
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            self.frames.append(frame)
            self.frames_cropped.append(cv2.resize((crop(frame)),(64,64)))
            run += 1
            progress_bar.progress(run/self.frame_count)
        self.cap.release()
    
    def create_subs(self):
        #divide subs to see
        duration = 0
        sub = ""
        subs = []
        for i in self.phone_timestamp:
            duration = duration + i["duration"]
            sub = sub + i["phone"] + " "
            if duration > 1.12:
                subs.append(sub)
                duration = 0
                sub = ""
        times = 1
        self.subbed_frames = []
        print(subs)
        font = cv2.FONT_HERSHEY_SIMPLEX
        for _frame in range(len(self.frames)):
            frame = self.frames[_frame]
            oneframetime = self.lenght/self.frame_count
            if (_frame+1)*oneframetime > 1.12*times:
                times += 1
            if len(subs) > times-1:
                frame = cv2.putText(frame, subs[times-1], (100,100), font, 3, (255, 255, 255))
            self.subbed_frames.append(frame)

    def create_vid(self):
        size = self.frames[0].shape[1], self.frames[0].shape[0]
        out = cv2.VideoWriter("output.mp4", cv2.VideoWriter_fourcc(*'DIVX'), self.fps, size)
        for img in self.frames:
            out.write(cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
        out.release()

Overwriting vidtoimage.py


In [ ]:
#@title Classify.py
%%writefile classify.py
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.models
import numpy as np

top = 2
classes = np.array(['aa', 'ae', 'ah', 'ao', 'aw', 'ay', 'b', 'ch', 'd', 'dh', 'eh',
       'er', 'ey', 'f', 'g', 'hh', 'ih', 'iy', 'jh', 'k', 'l', 'm', 'n',
       'ng', 'none', 'ow', 'oy', 'p', 'r', 's', 'sh', 't', 'th', 'uh',
       'uw', 'v', 'w', 'y', 'z', 'zh'], dtype='<U4')

pronunciations = {}
for line in open("cmudict-0.7b", 'r', errors='replace').readlines():
  line = line.strip()
  if line.startswith(";"): continue
  word, phones = line.split("  ")
  phones = phones.split(" ")
  for p in range(len(phones)):
    phones[p] = phones[p].replace("0", "")
    phones[p] = phones[p].replace("1", "")
    phones[p] = phones[p].replace("2", "")
    phones[p] = phones[p].lower()
  word = word.rstrip("(0123)").lower()
  if word not in pronunciations:
    pronunciations[word] = []
  pronunciations[word].append(phones)

model = keras.models.load_model("Models/model_accuracy_35_loss_2.2")

def predict(video):
    data = []
    for i in range(video.frame_count//video.SEQUENCE_LENGHT):
        sequence =[]
        for a in range(video.SEQUENCE_LENGHT):
            sequence.append(video.frames_cropped[i+a])
        data.append(np.array([sequence]))
    for _seq in range(len(data)):
        prediction = model.predict((data[_seq]))
        sort = np.sort(prediction[0])
        video.prediction[_seq] = (classes[np.where(prediction[0]==sort[-(top)])[0][0]])
    for _frames in range(len(video.prediction)):
        if len(video.phone_timestamp)!=0 and video.prediction[_frames] == video.phone_timestamp[-1]["phone"]:
            video.phone_timestamp[-1]["time_end"] = video.timepersequence*(_frames+1)
            video.phone_timestamp[-1]["duration"] = video.phone_timestamp[-1]["time_end"] - video.phone_timestamp[-1]["time_start"]
        else:
            pred = {}
            pred["phone"] = video.prediction[_frames]
            pred["time_start"] = video.timepersequence*_frames
            pred["time_end"] = video.timepersequence*(_frames+1)
            pred["duration"] = pred["time_end"] - pred["time_start"]
            video.phone_timestamp.append(pred)
    possible_words = [{"start_time":0}]
    word = []
    for _dict in range(len(video.phone_timestamp)):
        dict = video.phone_timestamp[_dict]
        if round(dict["duration"]) == round(video.timepersequence):
            word.append(dict["phone"])
        elif round(dict["duration"]) > round(video.timepersequence):
            word.append(dict["phone"])
            possible_words[-1]["time_end"] = video.phone_timestamp[_dict-1]["time_end"]
            possible_words[-1]["word"] = word
            possible_words.append({})
            possible_words[-1]["time_start"] = dict["time_start"]
            word = []
    words = []
    video.possible_words = possible_words

Overwriting classify.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

2022-06-21 15:30:30.701 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
npx: installed 22 in 2.319s

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.199.14.102:8501

your url is: https://ready-turtles-argue-35-199-14-102.loca.lt
2022-06-21 15:30:56.327439: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2022-06-21 15:31:35.360 Uncaught app exception
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/streamlit/scriptrunner/script_runner.py", line 554, in _run_script
    exec(code, module.__dict__)
  File "app.py", line 52, in <module>
    main()
  File "app.py", line 18, in main
    id = user_input.split("/")[5]
IndexError: list index out of range
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was de